In [249]:
import datetime as dt

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import _fetch_batch, fetch_data, fetch_spot_data

In [250]:
async def get_expiry(f_today):

    if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
        f_expiry = dt.date(2024, 1, 25)
    elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
        f_expiry = dt.date(2024, 1, 31)
    elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
        f_expiry = dt.date(2024, 2, 29)
    elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
        f_expiry = dt.date(2024, 2, 27)
    elif f_today < dt.date(2023, 9, 1):
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
    elif f_today >= dt.date(2023, 9, 1):
        if f_today.day < 24:
            days_to_wednesday = (2 - f_today.weekday()) % 7
            nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
            f_expiry = nearest_wednesday
            if nse.valid_days(
                start_date=nearest_wednesday, end_date=nearest_wednesday
            ).empty:
                f_expiry = nearest_wednesday - dt.timedelta(days=1)
        else:
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(
                start_date=nearest_thursday, end_date=nearest_thursday
            ).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_expiry_finnifty(f_today):

    days_to_thursday = (1 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry

async def get_expiry_nifty(f_today):

    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry

async def get_expiry_midcpnifty(f_today):

    days_to_thursday = (0 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [251]:
# bnf_pandas = pd.read_csv("../data/bnf.csv")
bnf_pandas = pd.read_csv("../data/bnf_1hr_tv (9).csv")

In [252]:
bnf_pandas["datetime"] = pd.to_datetime(bnf_pandas["time"])
# bnf_pandas["datetime"] = pd.to_datetime(bnf_pandas["datetime"])
bnf_pandas["datetime"] = bnf_pandas["datetime"].dt.tz_localize(None)
bnf_pandas = bnf_pandas[bnf_pandas["datetime"].dt.year >= 2017]

bnf = pl.DataFrame(bnf_pandas)
print(type(bnf))
bnf = bnf.with_columns([pl.col("datetime").alias("index")]).drop("datetime")
bnf = bnf.with_columns(pl.col("index").alias("datetime"))
bnf.tail()

<class 'polars.dataframe.frame.DataFrame'>


time,open,high,low,close,index,datetime
str,f64,f64,f64,f64,datetime[ns],datetime[ns]
"""2024-07-25T11:…",50736.35,50879.15,50696.55,50839.65,2024-07-25 11:15:00,2024-07-25 11:15:00
"""2024-07-25T12:…",50837.25,50879.55,50778.9,50833.25,2024-07-25 12:15:00,2024-07-25 12:15:00
"""2024-07-25T13:…",50831.35,50921.7,50776.1,50890.4,2024-07-25 13:15:00,2024-07-25 13:15:00
"""2024-07-25T14:…",50894.4,50936.25,50841.45,50897.9,2024-07-25 14:15:00,2024-07-25 14:15:00
"""2024-07-25T15:…",50898.55,50898.55,50853.8,50886.55,2024-07-25 15:15:00,2024-07-25 15:15:00


In [253]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("o").first().alias("o"),
                pl.col("h").max().alias("h"),
                pl.col("l").min().alias("l"),
                pl.col("c").last().alias("c"),
                pl.col("v").sum().alias("v"),
            ]
        )
    )


# ohlc_resampled = resample(bnf, '5m', pd.Timedelta(minutes=0))

# bnf_1hr = ohlc_resampled
# bnf_final = bnf_1hr.to_pandas()
# bnf_final['datetime'] = pd.to_datetime(bnf_final['datetime'])
# bnf_final
# bnf_1hr

In [254]:
def calculate_vwap(df):

    df = df.sort_values(by=['datetime'])
    df['date'] = pd.to_datetime(df['datetime']).dt.date
    df['typical_price'] = (df['h'] + df['l'] + df['c']) / 3
    df['price_volume'] = df['typical_price'] * df['v']
    df['cumulative_price_volume'] = df.groupby('date')['price_volume'].cumsum()
    df['cumulative_volume'] = df.groupby('date')['v'].cumsum()
    df['vwap'] = df['cumulative_price_volume'] / df['cumulative_volume']
    df = df.drop(columns=['date', 'typical_price', 'price_volume', 'cumulative_price_volume', 'cumulative_volume'])
    # print(df.to_string())
    return df

In [255]:
data_bnf = bnf_pandas.copy()
trading_days_set = set(data["datetime"].dt.date)
data_bnf.tail()

,time,open,high,low,close,datetime
21651,2024-07-25T11:15:00+05:30,50736.3500,50879.1500,50696.5500,50839.6500,2024-07-25 11:15:00
21652,2024-07-25T12:15:00+05:30,50837.2500,50879.5500,50778.9000,50833.2500,2024-07-25 12:15:00
21653,2024-07-25T13:15:00+05:30,50831.3500,50921.7000,50776.1000,50890.4000,2024-07-25 13:15:00
21654,2024-07-25T14:15:00+05:30,50894.4000,50936.2500,50841.4500,50897.9000,2024-07-25 14:15:00
21655,2024-07-25T15:15:00+05:30,50898.5500,50898.5500,50853.8000,50886.5500,2024-07-25 15:15:00


In [256]:
# GLOBAL VARIABLES

INSTRUMENT_VWAP = "BANKNIFTY"
INDEX_VWAP = "bnf"

# INSTRUMENT = "MIDCPNIFTY"
# INDEX = "midcpnifty"

# INSTRUMENT = "FINNIFTY"
# INDEX = "finnifty"

# INSTRUMENT = "NIFTY"
# INDEX = "nifty"

PORTFOLIO_VALUE = 50_00_000
INDEX_LEV = 6

In [257]:
async def option_selling(
    df, index, strike, spot_open, asset_class, expiry
):
    trade_book = []
    signal_entry_price = 100000
    signal_initial_sl = 0
    remark = ""
    
    in_trade = False
    eod_exit_flag = False

    for i in range(0, len(df)):
        subset_df = df[:i+1]
        points = 0
        
        current_candle_open = df.iloc[i]["o"]
        current_candle_high = df.iloc[i]["h"]
        current_candle_low = df.iloc[i]["l"]
        current_candle_close = df.iloc[i]["c"]
        current_candle_time = df.iloc[i]['datetime']
        current_vwap = df.iloc[i]["vwap"]

        previous_candle_open = df.iloc[i-1]["o"]
        previous_candle_high = df.iloc[i-1]["h"]
        previous_candle_low = df.iloc[i-1]["l"]
        previous_candle_close = df.iloc[i-1]["c"]
        previous_candle_time = df.iloc[i-1]['datetime']
        previous_vwap = df.iloc[i-1]["vwap"]

        if not in_trade:
            if (previous_candle_close <= previous_vwap) and (current_candle_close <= current_vwap) and (current_candle_low <= previous_candle_low):
                #Entry Triggered
                in_trade = True
                entry_price = current_candle_close
                stop_loss = subset_df['h'].max()
                signal_time = previous_candle_time
                entry_time = current_candle_time
                # qty = PORTFOLIO_VALUE * 0.005 / abs(stop_loss - entry_price)
                qty = int(round((PORTFOLIO_VALUE * INDEX_LEV / strike) / 15) * 15)
                vwap_at_entry = current_vwap

        if in_trade:
            if current_candle_high >= stop_loss:
                #SL Hit
                in_trade = False
                exit_time = current_candle_time
                exit_price = stop_loss
                points = entry_price - exit_price
                remark = 'SL Hit'
                # vwap_at_exit = current_vwap

            elif current_candle_time.time() >= dt.time(15, 20) and not eod_exit_flag:
                # EOD Exit

                in_trade = False
                exit_time = current_candle_time
                exit_price = current_candle_close
                points = entry_price - exit_price
                remark = 'EOD Exit'
                # vwap_at_exit = current_vwap

        if points:
            if (entry_price - stop_loss < 0): # To Avoid inf QTY trades
                slippage = 0.01 * (entry_price + exit_price)
                final_points = points - slippage
                dte = (expiry - entry_time.date()).days
                trade = {
                    "Index": index,
                    'Spot Open': spot_open,
                    "Strike": strike,
                    "Option Type": asset_class,
                    "Expiry": expiry,
                    "DTE": dte,
                    "Signal Generated At": signal_time,
                    "Trade Type": "SELL",
                    "Entry Time": entry_time,
                    'VWAP': vwap_at_entry,
                    "Entry Price": entry_price,
                    "Initial SL": stop_loss,
                    "Exit Time": exit_time,
                    "Exit Price": exit_price,
                    "Points Captured": points,
                    "Slippages": slippage,
                    "After Costs": final_points,
                    "PnL": final_points * qty,
                    "Remarks": remark,
                    "Qty": qty,
                    "ROI%": (final_points * qty / PORTFOLIO_VALUE) * 100,
                    "Trade Year": entry_time.year,
                    "Trade Month": entry_time.month,
                }
                trade_book.append(trade)
                points = 0
                in_trade = False
                remark = ""
                eod_exit_flag = False

    return pd.DataFrame(trade_book)

In [258]:
async def option_selling_low_break(
    df, index, strike, spot_open, asset_class, expiry
):
    trade_book = []
    signal_entry_price = 100000
    signal_initial_sl = 0
    remark = ""
    
    in_trade = False
    eod_exit_flag = False

    for i in range(0, len(df)):
        subset_df = df[:i+1]
        points = 0
        
        current_candle_open = df.iloc[i]["o"]
        current_candle_high = df.iloc[i]["h"]
        current_candle_low = df.iloc[i]["l"]
        current_candle_close = df.iloc[i]["c"]
        current_candle_time = df.iloc[i]['datetime']
        current_vwap = df.iloc[i]["vwap"]

        previous_candle_open = df.iloc[i-1]["o"]
        previous_candle_high = df.iloc[i-1]["h"]
        previous_candle_low = df.iloc[i-1]["l"]
        previous_candle_close = df.iloc[i-1]["c"]
        previous_candle_time = df.iloc[i-1]['datetime']
        previous_vwap = df.iloc[i-1]["vwap"]

        if not in_trade:
            if (previous_candle_close <= previous_vwap) and (current_candle_low <= previous_candle_low):
                #Entry Triggered
                in_trade = True
                entry_price = previous_candle_low
                stop_loss = subset_df['h'].max()
                signal_time = previous_candle_time
                entry_time = current_candle_time
                # qty = PORTFOLIO_VALUE * 0.005 / abs(stop_loss - entry_price)
                qty = int(round((PORTFOLIO_VALUE * INDEX_LEV / strike) / 15) * 15)
                vwap_at_entry = current_vwap

        if in_trade:
            if current_candle_high >= stop_loss:
                #SL Hit
                in_trade = False
                exit_time = current_candle_time
                exit_price = stop_loss
                points = entry_price - exit_price
                remark = 'SL Hit'
                # vwap_at_exit = current_vwap

            elif current_candle_time.time() >= dt.time(15, 20) and not eod_exit_flag:
                # EOD Exit

                in_trade = False
                exit_time = current_candle_time
                exit_price = current_candle_close
                points = entry_price - exit_price
                remark = 'EOD Exit'
                # vwap_at_exit = current_vwap

        if points:
            if (entry_price - stop_loss < 0): # To Avoid inf QTY trades
                slippage = 0.01 * (entry_price + exit_price)
                final_points = points - slippage
                dte = (expiry - entry_time.date()).days
                trade = {
                    "Index": index,
                    'Spot Open': spot_open,
                    "Strike": strike,
                    "Option Type": asset_class,
                    "Expiry": expiry,
                    "DTE": dte,
                    "Signal Generated At": signal_time,
                    "Trade Type": "SELL",
                    "Entry Time": entry_time,
                    'VWAP': vwap_at_entry,
                    "Entry Price": entry_price,
                    "Initial SL": stop_loss,
                    "Exit Time": exit_time,
                    "Exit Price": exit_price,
                    "Points Captured": points,
                    "Slippages": slippage,
                    "After Costs": final_points,
                    "PnL": final_points * qty,
                    "Remarks": remark,
                    "Qty": qty,
                    "ROI%": (final_points * qty / PORTFOLIO_VALUE) * 100,
                    "Trade Year": entry_time.year,
                    "Trade Month": entry_time.month,
                }
                trade_book.append(trade)
                points = 0
                in_trade = False
                remark = ""
                eod_exit_flag = False

    return pd.DataFrame(trade_book)

In [267]:
async def trade(df):

    start_date = dt.date(2024, 3, 1)
    end_date = dt.date(2024, 3, 29)

    current_date = start_date

    combined_trades = pd.DataFrame()
    total_trades = pd.DataFrame()

    while(current_date <= end_date):
        if current_date in trading_days_set:
            print(current_date)

            spot_check_time = dt.time(9, 15)
            starting_time = dt.time(9, 15)
            ending_time = dt.time(15, 30)

            filtered_df = df.loc[(df['datetime'].dt.date == current_date) & (df['datetime'].dt.time >= spot_check_time)]
            if not filtered_df.empty:
                spot_open = filtered_df["open"].iloc[0]
            else:
                current_date += dt.timedelta(days=1)
                continue
                
            # print(spot_open)
            spot_atm = int(round(spot_open / 100) * 100)
            # print(spot_atm)
            nearest_expiry = await get_expiry(current_date)
            # print(current_date, nearest_expiry)

            ce_df = await fetch_data(
                index=INDEX_VWAP,
                start_date=current_date,
                start_time=starting_time,
                end_date=current_date,
                end_time=ending_time,
                strike=spot_atm,
                asset_class="C",
                expiry=nearest_expiry,
            )
            
            if not isinstance(ce_df, str) and (ce_df is not None):
                # print(ce_df.head())
                # print(ce_df.tail())
                ce_df = ce_df.select(["datetime", "o", "h", "l", "c", "v"])
                ce_df = resample(ce_df, "3m")
                ce_df_pandas = ce_df.to_pandas()
                ce_df_pandas = calculate_vwap(ce_df_pandas)
                # ce_df = generate_signals2(ce_df_pandas)
                # print(ce_df.tail())
            else:
                # print("ELSE CONDITION TRIGGERED -> CE Option")
                # print(ce_df)
                current_date += dt.timedelta(days=1)
                continue

            pe_df = await fetch_data(
                index=INDEX_VWAP,
                start_date=current_date,
                start_time=starting_time,
                end_date=current_date,
                end_time=ending_time,
                strike=spot_atm,
                asset_class="P",
                expiry=nearest_expiry,
            )
            
            if not isinstance(pe_df, str) and (pe_df is not None):
                # print(pe_df.head())
                # print(pe_df.tail())
                pe_df = pe_df.select(["datetime", "o", "h", "l", "c", "v"])
                pe_df = resample(pe_df, "3m")
                pe_df_pandas = pe_df.to_pandas()
                pe_df_pandas = calculate_vwap(pe_df_pandas)
                # pe_df = generate_signals2(pe_df_pandas)
                # print(pe_df.tail())
            else:
                # print("ELSE CONDITION TRIGGERED -> PE Option")
                # print(pe_df)
                current_date += dt.timedelta(days=1)
                continue

            # print(
            #     ce_df_pandas.to_string(), "\n", pe_df_pandas.to_string()
            # )

            # print('WORKING ON CE TRADES')
            ce_trades = await option_selling_low_break(
                df=ce_df_pandas,
                index=INSTRUMENT_VWAP,
                strike=spot_atm,
                spot_open=spot_open,
                asset_class="C",
                expiry=nearest_expiry,
            )
            # print('WORKING ON PE TRADES')
            pe_trades = await option_selling_low_break(
                df=pe_df_pandas,
                index=INSTRUMENT_VWAP,
                strike=spot_atm,
                spot_open=spot_open,
                asset_class="P",
                expiry=nearest_expiry,
            )
            combined_trades = pd.concat([combined_trades, ce_trades], ignore_index=True)
            combined_trades = pd.concat([combined_trades, pe_trades], ignore_index=True)
            # print(combined_trades.to_string())
            
        current_date += dt.timedelta(days=1)

    total_trades = pd.concat([total_trades, combined_trades], ignore_index=True)

    return total_trades

In [268]:
def generate_stats(tb):
    stats_df8 = pd.DataFrame(
        index=range(2017, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            # "Variation",
        ],
    )
    combined_df_sorted = tb
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2017, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        # variation = f'{signal_ma} , {trailing_ma}, {time_of_day}'
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            # variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    # overall_variation = variation
    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        # overall_variation,
    ]
    return stats_df8

In [269]:
tb = await trade(data_bnf)

2024-03-01
2024-03-02
2024-03-04
2024-03-05
2024-03-06
2024-03-07
2024-03-11
2024-03-12
2024-03-13
2024-03-14
2024-03-15
2024-03-18
2024-03-19
2024-03-20
2024-03-21
2024-03-22
2024-03-26
2024-03-27
2024-03-28


In [270]:
# tb

In [271]:
tb['ROI%'].sum()
# tb['After Costs'].sum()

KeyError: 'ROI%'

In [272]:
stats_15min = generate_stats(tb)
stats_15min

KeyError: 'Trade Year'

In [226]:
stats_1min = generate_stats(tb)
stats_1min

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,36.0881,775,39.3548,1.1523,-0.6710,-28.1555,1.2817
2018,28.3310,758,37.9947,1.5476,-0.8880,-18.5128,1.5303
2019,67.6165,773,39.0686,1.8820,-1.0632,-26.6596,2.5363
2020,97.2568,765,41.0458,3.7339,-2.3840,-61.7188,1.5758
2021,48.5677,769,41.3524,2.2339,-1.4674,-23.3752,2.0777
2022,-89.5930,796,37.5628,1.9156,-1.3327,-91.3630,-0.9806
2023,-28.8132,756,37.1693,1.2987,-0.8289,-40.9109,-0.7043
2024,3.7555,146,35.6164,1.8954,-1.0086,-11.8269,0.3175
Overall,163.2094,5538.0000,38.9852,1.9845,-1.2197,-127.6464,1.2786


In [222]:
stats_5min = generate_stats(tb)
stats_5min

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,43.2562,608,46.7105,1.0352,-0.7739,-15.0436,2.8754
2018,6.9203,632,42.4051,1.4262,-1.0310,-21.5291,0.3214
2019,65.0969,627,45.6140,1.6693,-1.2092,-21.0989,3.0853
2020,66.6394,632,46.9937,3.3551,-2.7756,-63.6803,1.0465
2021,-20.2383,638,46.7085,1.9714,-1.7874,-33.4010,-0.6059
2022,-130.7398,664,41.7169,1.7207,-1.5694,-134.4516,-0.9724
2023,-51.9481,613,43.7194,1.1128,-1.0150,-62.0124,-0.8377
2024,14.8112,106,46.2264,1.6644,-1.1709,-9.2832,1.5955
Overall,-6.2023,4520.0000,44.8451,1.7731,-1.4441,-216.3947,-0.0287


In [216]:
stats = generate_stats(tb)
stats

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,56.3223,651,45.0077,1.0842,-0.7300,-13.2041,4.2655
2018,14.8771,670,41.6418,1.4725,-1.0126,-20.4691,0.7268
2019,81.8485,661,45.6884,1.7274,-1.2251,-17.3236,4.7247
2020,83.8766,681,45.0808,3.4645,-2.6196,-57.8172,1.4507
2021,-14.7237,678,43.6578,2.1062,-1.6705,-31.3787,-0.4692
2022,-89.8603,695,41.2950,1.8024,-1.4881,-94.3914,-0.9520
2023,-37.4110,650,42.4615,1.1952,-0.9820,-51.3070,-0.7292
2024,15.4032,120,41.6667,1.8023,-1.0673,-9.1400,1.6852
Overall,110.3326,4806.0000,43.4873,1.8538,-1.3859,-155.3313,0.7103
